docker run -d \
  --name qdrant \
  -p 6333:6333 \
  qdrant/qdrant

In [19]:
from langchain_community.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from tqdm import tqdm
import os

HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder and its subfolders
    for root, _, files in os.walk(data_path):
        for filename in files:
            if filename.endswith('.txt'):
                file_path = os.path.join(root, filename)
                loader = TextLoader(file_path, encoding='utf-8')
                docs.extend(loader.load())

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=True
    )

    chunk_size = 512
    chunk_overlap = 0

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunked_docs = []

    for doc in docs:
        md_header_splits = markdown_splitter.split_text(doc.page_content)
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs




/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/data/sotaysinhvien'
docs = load_and_chunk_data(data_path)

In [29]:
print(len(docs))
for doc in docs: 
    print(doc)


407
page_content='Địa chỉ: Tầng 4 - S4, Tầng 3 - S4, Tầng 5 - S4, Tầng 3 - S4
Điện thoại: 0914539578 (đ/c Chinh), 0988116875 (đ/c Thể)' metadata={'Header 1': '1. Danh bạ các Phòng, Ban và Khoa', 'Header 2': '1.1. Phòng Chính trị - Ban Thanh niên - Ban Bảo vệ - Bộ phận tiếp nhận giải quyết thủ tục hành chính'}
page_content='Địa chỉ: Tầng 7 - S4, Tầng 7 - S4, Tầng 1 - S4
Điện thoại: 069515221, 069698263, 0979731766 (đ/c Mạnh)' metadata={'Header 1': '1. Danh bạ các Phòng, Ban và Khoa', 'Header 2': '1.2. Phòng Đào tạo - Ban Kế hoạch - Ban Quản lý đào tạo'}
page_content='Địa chỉ: Tầng 5 - S4' metadata={'Header 1': '1. Danh bạ các Phòng, Ban và Khoa', 'Header 2': '1.3. Phòng Hậu cần'}
page_content='Địa chỉ: Tầng 4 - S4' metadata={'Header 1': '1. Danh bạ các Phòng, Ban và Khoa', 'Header 2': '1.4. Văn phòng'}
page_content='Địa chỉ: Tầng 5, 6 - S4' metadata={'Header 1': '1. Danh bạ các Phòng, Ban và Khoa', 'Header 2': '1.5. Phòng Kỹ thuật'}
page_content='Địa chỉ: Tầng 6 - S4' metadata={'Header 

In [30]:
from dotenv import load_dotenv
load_dotenv()


url = os.getenv("URL_QDRANT")
api_key = os.getenv("API_QDRANT")
qdrant = QdrantVectorStore.from_documents(
    docs,
    HF_EMBEDDING,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="sotaysinhvien_db",
)